# 1. Librerias

In [15]:
# Instalamos la librería 'geopy'
!pip install geopy

     ------------------------------------ 119.8/119.8 kB 876.6 kB/s eta 0:00:00
     -------------------------------------- 40.3/40.3 kB 477.5 kB/s eta 0:00:00


In [16]:
import pandas as pd

from geopy.geocoders import Nominatim
import numpy as np

# 2. Importamos CSV

In [5]:
ruta_archivo_csv = '../01_data/datos_locales.csv'

df_locales = pd.read_csv(ruta_archivo_csv)

df_locales.head(5)

,local,direccion,tipo_local,valoracion_google,numero_reseñas_google,enlace_web,puntuacion_entradas_com,numero_entradas_com,puntuacion_facebook,numero_puntuacion_facebook
0,Teatro Príncipe Gran Vía,"C. de las Tres Cruces, 8, 28013 Madrid",teatro,4.2,2601,http://www.teatroprincipegranvia.es/,3.7,823.0,4.3,89.0
1,El Golfo Comedy Club,"C. de las Huertas, 57, 28014 Madrid",club de comedia,4.7,1082,https://elgolfocomedy.com/,NaN,NaN,NaN,NaN
2,Teatro Arlequín Gran Vía,"C. de San Bernardo, 5, 28013 Madrid",teatro,4.2,5264,http://teatroarlequingranvia.com/,3.7,329.0,4.6,103.0
3,Teatro Capitol Gran Vía,"C/ Gran Vía, 41, 28013 Madrid",teatro,4.2,4902,http://www.capitolgranvia.com/,4.0,577.0,NaN,NaN
4,Estación Malasaña,"C. del Pez, 16, 28004 Madrid",bar,4.5,344,https://www.estacionmalasana.es/,3.9,23.0,NaN,NaN


# 3. Limpiamos duplicados

En teoría, no puede haber locales duplicados, así como direcciones duplicadas.

In [7]:
df_locales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   local                       48 non-null     object 
 1   direccion                   48 non-null     object 
 2   tipo_local                  48 non-null     object 
 3   valoracion_google           48 non-null     float64
 4   numero_reseñas_google       48 non-null     int64  
 5   enlace_web                  44 non-null     object 
 6   puntuacion_entradas_com     22 non-null     float64
 7   numero_entradas_com         22 non-null     float64
 8   puntuacion_facebook         14 non-null     float64
 9   numero_puntuacion_facebook  14 non-null     float64
dtypes: float64(5), int64(1), object(4)
memory usage: 3.9+ KB


## 3.1 Comprobamos la columna de locales

In [6]:
len(df_locales['local'].unique())

47

In [8]:
# Tenemos 48 filas y solo salen 47 valores únicos, por lo que hay un duplicado, comprobamos:
filas_duplicadas = df_locales[df_locales.local.duplicated(keep=False)]
filas_duplicadas

,local,direccion,tipo_local,valoracion_google,numero_reseñas_google,enlace_web,puntuacion_entradas_com,numero_entradas_com,puntuacion_facebook,numero_puntuacion_facebook
18,Cine Palacio de la Prensa,"Pl. del Callao, 4, 28013 Madrid",cine,4.1,2503,http://www.palaciodelaprensa.com/,NaN,NaN,NaN,NaN
29,Cine Palacio de la Prensa,"Pl. del Callao, 4, 28013 Madrid",cine,4.1,2503,http://www.palaciodelaprensa.com/,NaN,NaN,NaN,NaN


In [9]:
# Vemos que efectivamente se trata del mismo local, por lo que eliminamos duplicados de la columna.
df_locales = df_locales.drop_duplicates()

## 3.2 Comprobamos la columna de direcciones

In [11]:
len(df_locales['direccion'].unique())

45

In [12]:
# Ahora tenemos 47 filas y solo salen 45 valores únicos, por lo que hay dos duplicados, comprobamos:
filas_duplicadas = df_locales[df_locales.direccion.duplicated(keep=False)]
filas_duplicadas

,local,direccion,tipo_local,valoracion_google,numero_reseñas_google,enlace_web,puntuacion_entradas_com,numero_entradas_com,puntuacion_facebook,numero_puntuacion_facebook
12,Pequeño Teatro Gran Vía de Madrid,"C/ Gran Vía, 66, 28013 Madrid",teatro,4.5,1975,https://entradas.gruposmedia.com/entradas/es/e...,NaN,NaN,NaN,NaN
22,Cervecería Valverde Comedy Club,"C. de Valverde, 4, 28004 Madrid",club de comedia,4.0,1085,NaN,3.0,1.000,NaN,NaN
36,Teatro EDP Gran Vía,"C/ Gran Vía, 66, 28013 Madrid",teatro,4.2,9571,https://gruposmedia.com/teatro-edp-gran-via/,3.9,3.577,NaN,NaN
40,Cafetería Valverde,"C. de Valverde, 4, 28004 Madrid",bar,4.0,1085,NaN,NaN,NaN,NaN,NaN


Buscamos las direcciones duplicadas en google, donde vemos que las dos cervecerías valverde son la misma y que el teatro de la gran vía está separado en dos salas diferentes. De esta manera, eliminamos una de las cafeterías valverdes y los teatros los dejamos.

In [13]:
df_locales.drop(index=40, inplace=True)

# 4. Creación de columnas para geolocalizaciones

In [21]:
geolocator = Nominatim(user_agent="my_app")

# creamos una funcion para obtener la latitud y longitud de un pais
def get_lat_lon(direccion):
    """ la funcion recibe una dirección y devuelve la latitud y longitud


    Args:
        direccion (str): nombre del pais a buscar
    Returns:
        tuple: devuelve una tuple con la latitud y longitud
    """
    try:
        location = geolocator.geocode(direccion)
        return location.latitude, location.longitude
    except:
        return np.nan, np.nan

In [19]:
lista_direcciones = df_locales['direccion'].tolist()
lista_direcciones

['C. de las Tres Cruces, 8, 28013 Madrid',
 'C. de las Huertas, 57, 28014 Madrid',
 'C. de San Bernardo, 5, 28013 Madrid',
 'C/ Gran Vía, 41, 28013 Madrid',
 'C. del Pez, 16, 28004 Madrid',
 'C. de Alcalá, 20, 28014 Madrid',
 'Calle de Fuencarral, 136, 28010 Madrid',
 'C. de Trafalgar, 15, 28010 Madrid',
 'C. de las Aguas, 8, 28005 Madrid',
 'C. de las Navas de Tolosa, 3, 28013 Madrid',
 'Cuesta de Sto. Domingo, 22, 28013 Madrid',
 'C. de Luchana, 38, 28010 Madrid',
 'C/ Gran Vía, 66, 28013 Madrid',
 'C. de Tomás Bretón, 20, 28045 Madrid',
 'Cta. de San Vicente, 44, 28008 Madrid',
 'C. de San Roque, 14, 28004 Madrid',
 'Av. de Brasil, 17, 28020 Madrid',
 'Cra de S. Jerónimo, 24, 28014 Madrid',
 'Pl. del Callao, 4, 28013 Madrid',
 'Calle del Dr. Urquiola, 23, 28025 Madrid',
 'C. de las Tres Cruces, 12, 28013 Madrid',
 'C/ Gran Vía, 70, 28013 Madrid',
 'C. de Valverde, 4, 28004 Madrid',
 'Calle de Manuel Silvela, 6, 28010 Madrid',
 'C. de García Luna, 13, 28002 Madrid',
 'C. de la Lechug

In [22]:
diccionario_de_coordenadas = {}
for direccion in lista_direcciones:
    localoca = get_lat_lon(direccion)
    diccionario_de_coordenadas[direccion] = localoca
    print(direccion, localoca)

C. de las Tres Cruces, 8, 28013 Madrid (40.4195185, -3.7026612)
C. de las Huertas, 57, 28014 Madrid (40.41328525, -3.6964122018327306)
C. de San Bernardo, 5, 28013 Madrid (40.4211706, -3.7082023)
C/ Gran Vía, 41, 28013 Madrid (40.4683282, -3.8707172)
C. del Pez, 16, 28004 Madrid (40.4236731, -3.7051859)
C. de Alcalá, 20, 28014 Madrid (40.41762175, -3.6989692917634684)
Calle de Fuencarral, 136, 28010 Madrid (40.4315549, -3.7029393365099326)
C. de Trafalgar, 15, 28010 Madrid (40.4317969, -3.7008063)
C. de las Aguas, 8, 28005 Madrid (nan, nan)
C. de las Navas de Tolosa, 3, 28013 Madrid (40.4191439, -3.7065685)
Cuesta de Sto. Domingo, 22, 28013 Madrid (40.4197052, -3.7085243)
C. de Luchana, 38, 28010 Madrid (40.4318686, -3.6982529)
C/ Gran Vía, 66, 28013 Madrid (40.4683282, -3.8707172)
C. de Tomás Bretón, 20, 28045 Madrid (40.3973457, -3.6938624)
Cta. de San Vicente, 44, 28008 Madrid (nan, nan)
C. de San Roque, 14, 28004 Madrid (40.4225962, -3.7046498)
Av. de Brasil, 17, 28020 Madrid (40.4

In [41]:
lista_direcciones_corregir = []
for direccion, coordenadas in diccionario_de_coordenadas.items():
    if np.any(np.isnan(coordenadas)):
        lista_direcciones_corregir.append(direccion)

print("Direcciones que necesitan corrección:", lista_direcciones_corregir)

Direcciones que necesitan corrección: ['C. de las Aguas, 8, 28005 Madrid', 'Cta. de San Vicente, 44, 28008 Madrid', 'Pl. del Callao, 4, 28013 Madrid', 'Corre. Baja de San Pablo, 39-1, 28004 Madrid', 'C. Cruz de Guadalajara, 4, 28805 Alcalá de Henares, Madrid', 'Corre. Baja de San Pablo, 15, 28004 Madrid', 'C. el Electrodo, 50, 28522 Rivas-Vaciamadrid, Madrid', 'P.º del Gral. Martínez Campos, 9, 28010 Madrid', 'C. Real, 71, 28703 San Sebastián de los Reyes, Madrid']


In [52]:
# Diccionario de correcciones
correcciones = {
    'C. de las Aguas, 8, 28005 Madrid': 'Calle de las Aguas, 8, 28005 Madrid',
    'Cta. de San Vicente, 44, 28008 Madrid': 'Cuesta de San Vicente, 44, 28008 Madrid',
    'Pl. del Callao, 4, 28013 Madrid': 'Plaza Callao, 4, 28013 Madrid',
    'Corre. Baja de San Pablo, 39-1, 28004 Madrid': 'Corredera Baja de San Pablo, 39-1, 28004 Madrid',
    'C. Cruz de Guadalajara, 4, 28805 Alcalá de Henares, Madrid':'Calle Cruz de Guadalajara, 4, 28805 Alcalá de Henares, Madrid',
    'Corre. Baja de San Pablo, 15, 28004 Madrid':'Corredera Baja de San Pablo, 15, 28004 Madrid', 
    'C. el Electrodo, 50, 28522 Rivas-Vaciamadrid, Madrid':'Calle Electrodo, 50, 28522 Rivas-Vaciamadrid, Madrid', 
    'P.º del Gral. Martínez Campos, 9, 28010 Madrid':'Paseo General Martinez Campos, 9, 28010 Madrid', 
    'C. Real, 71, 28703 San Sebastián de los Reyes, Madrid':'Calle Real, 71, 28703 San Sebastián de los Reyes, Madrid'
}

# Lista para almacenar las direcciones corregidas
lista_direcciones_corregidas = []

# Corregimos las direcciones en la lista original
for direccion in lista_direcciones:
    direccion_corregida = correcciones.get(direccion, direccion)
    lista_direcciones_corregidas.append(direccion_corregida)

# Sacamos coordenadas
diccionario_de_coordenadas = {}
for direccion in lista_direcciones_corregidas:
    localoca = get_lat_lon(direccion)
    diccionario_de_coordenadas[direccion] = localoca
    print(direccion, localoca)

C. de las Tres Cruces, 8, 28013 Madrid (40.4195185, -3.7026612)
C. de las Huertas, 57, 28014 Madrid (40.41328525, -3.6964122018327306)
C. de San Bernardo, 5, 28013 Madrid (40.4211706, -3.7082023)
C/ Gran Vía, 41, 28013 Madrid (40.4683282, -3.8707172)
C. del Pez, 16, 28004 Madrid (40.4236731, -3.7051859)
C. de Alcalá, 20, 28014 Madrid (40.41762175, -3.6989692917634684)
Calle de Fuencarral, 136, 28010 Madrid (40.4315549, -3.7029393365099326)
C. de Trafalgar, 15, 28010 Madrid (40.4317969, -3.7008063)
Calle de las Aguas, 8, 28005 Madrid (40.410900600000005, -3.712216945685541)
C. de las Navas de Tolosa, 3, 28013 Madrid (40.4191439, -3.7065685)
Cuesta de Sto. Domingo, 22, 28013 Madrid (40.4197052, -3.7085243)
C. de Luchana, 38, 28010 Madrid (40.4318686, -3.6982529)
C/ Gran Vía, 66, 28013 Madrid (40.4683282, -3.8707172)
C. de Tomás Bretón, 20, 28045 Madrid (40.3973457, -3.6938624)
Cuesta de San Vicente, 44, 28008 Madrid (40.4208539, -3.7187511)
C. de San Roque, 14, 28004 Madrid (40.4225962, 

In [58]:

# Agregar manualmente las coordenadas para las direcciones específicas
coordenadas_manuales = {
    'C. Cruz de Guadalajara, 4, 28805 Alcalá de Henares, Madrid': (40.48500660617931, -3.3599625803074087),
    'C. Real, 71, 28703 San Sebastián de los Reyes, Madrid': (40.5471734, -3.6255753)
}

# Crear una lista para almacenar las coordenadas correspondientes a las direcciones en df_locales
coordenadas_lista = []

# Obtener las coordenadas para cada dirección en df_locales
for direccion in df_locales['direccion']:
    coordenadas = coordenadas_manuales.get(direccion)
    if coordenadas is None:
        coordenadas = diccionario_de_coordenadas.get(direccion, None)  # Intenta obtener las coordenadas exactas
        if coordenadas is None:
            direccion_corregida = correcciones.get(direccion)
            if direccion_corregida is not None:
                coordenadas = diccionario_de_coordenadas.get(direccion_corregida)
    coordenadas_lista.append(coordenadas)

# Agregar la lista de coordenadas como una nueva columna al DataFrame df_locales
df_locales['coordenadas'] = coordenadas_lista

df_locales.head()

,local,direccion,tipo_local,valoracion_google,numero_reseñas_google,enlace_web,puntuacion_entradas_com,numero_entradas_com,puntuacion_facebook,numero_puntuacion_facebook,coordenadas
0,Teatro Príncipe Gran Vía,"C. de las Tres Cruces, 8, 28013 Madrid",teatro,4.2,2601,http://www.teatroprincipegranvia.es/,3.7,823.0,4.3,89.0,"(40.4195185, -3.7026612)"
1,El Golfo Comedy Club,"C. de las Huertas, 57, 28014 Madrid",club de comedia,4.7,1082,https://elgolfocomedy.com/,NaN,NaN,NaN,NaN,"(40.41328525, -3.6964122018327306)"
2,Teatro Arlequín Gran Vía,"C. de San Bernardo, 5, 28013 Madrid",teatro,4.2,5264,http://teatroarlequingranvia.com/,3.7,329.0,4.6,103.0,"(40.4211706, -3.7082023)"
3,Teatro Capitol Gran Vía,"C/ Gran Vía, 41, 28013 Madrid",teatro,4.2,4902,http://www.capitolgranvia.com/,4.0,577.0,NaN,NaN,"(40.4683282, -3.8707172)"
4,Estación Malasaña,"C. del Pez, 16, 28004 Madrid",bar,4.5,344,https://www.estacionmalasana.es/,3.9,23.0,NaN,NaN,"(40.4236731, -3.7051859)"


# 5. Eliminamos columnas inecesarias

Las puntuaciones de Facebook y entradas.com son escasas y no nos dicen mucho, por lo que las eliminamos.

In [60]:
columnas_a_eliminar = [
    'puntuacion_entradas_com',
    'numero_entradas_com',
    'puntuacion_facebook',
    'numero_puntuacion_facebook'
]

# Eliminar las columnas del DataFrame
df_locales = df_locales.drop(columnas_a_eliminar, axis=1)
df_locales.head()

,local,direccion,tipo_local,valoracion_google,numero_reseñas_google,enlace_web,coordenadas
0,Teatro Príncipe Gran Vía,"C. de las Tres Cruces, 8, 28013 Madrid",teatro,4.2,2601,http://www.teatroprincipegranvia.es/,"(40.4195185, -3.7026612)"
1,El Golfo Comedy Club,"C. de las Huertas, 57, 28014 Madrid",club de comedia,4.7,1082,https://elgolfocomedy.com/,"(40.41328525, -3.6964122018327306)"
2,Teatro Arlequín Gran Vía,"C. de San Bernardo, 5, 28013 Madrid",teatro,4.2,5264,http://teatroarlequingranvia.com/,"(40.4211706, -3.7082023)"
3,Teatro Capitol Gran Vía,"C/ Gran Vía, 41, 28013 Madrid",teatro,4.2,4902,http://www.capitolgranvia.com/,"(40.4683282, -3.8707172)"
4,Estación Malasaña,"C. del Pez, 16, 28004 Madrid",bar,4.5,344,https://www.estacionmalasana.es/,"(40.4236731, -3.7051859)"


# 6. Guardamos CSV

In [61]:
# Especifico la ruta y el nombre del archivo CSV
ruta_archivo_csv = '../01_data/datos_locales_coordenadas.csv'

# Guardo el DataFrame en un archivo CSV
df_locales.to_csv(ruta_archivo_csv, index=False)